In [1]:
import numpy as np
import re
import json
import torch.utils.data
from torch.utils.data import DataLoader
from torchtext import data
import io,os
from torch import nn
from gensim.models import word2vec
from torch.nn.utils.rnn import pad_sequence
import visdom
from torch import optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [ ]:
BATCH_SIZE = 100
device = torch.device("cuda")

tag_to_ix = {'start_tag':0,'stop_tag':29,'pad_tag':30}

train_path = '/home/jongsu/Desktop/ijcnlp_dailydialog/train/'
test_path = '/home/jongsu/Desktop/ijcnlp_dailydialog/test/'
Desktop_path = '/home/jongsu/Desktop/'



START_TAG = 0
STOP_TAG = 1
PAD_TAG = 2


In [6]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


In [7]:

def sent_loader(sentence): #pre_process per sentence
    result = []
    for elem in sentence.split(' '):
        if elem != '':
            result = np.append(result, elem)
    return result,len(result)

def cal_dialogue(data):
    i = 0
    maxleng = 0
    while(i < len(data.Text)):
        if len(data.Text[i]) > maxleng:
            maxleng = len(data.Text[i])
        i = i + 1
    return maxleng

def batchload(dataset, repeat, BATCH_SIZE):
    
    '''
    while True:
        i = BATCH_SIZE
        while(i < len(dataset)):
            batch = []
            batch_seq = 0
            batchnum = a[i-BATCH_SIZE:i]
            while(batch_seq < BATCH_SIZE):
                batch.append(dataset[batchnum[batch_seq]])
                batch_seq = batch_seq + 1
            yield batch
            i = i + BATCH_SIZE

        batch = []
        batch_seq = 0
        batchnum = a[i-BATCH_SIZE:len(dataset)-1]
        while (batch_seq < len(batchnum)):
            batch.append(dataset[batchnum[batch_seq]])
            batch_seq = batch_seq + 1
            
        yield batch

        if repeat == False:
            break
    '''
    a = torch.arange(len(dataset)-1000,dtype=torch.int)
    while True:
        i = BATCH_SIZE
        while(i < len(dataset)-1000):
            batch = []
            batch_seq = 0
            batchnum = a[i-BATCH_SIZE:i]
            while(batch_seq < BATCH_SIZE):
                batch.append(dataset[batchnum[batch_seq]])
                batch_seq = batch_seq + 1
            yield batch
            i = i + BATCH_SIZE
        
        if repeat == False:
            break

    
def pad_batch(minibatch):
    i = 0
    new_batch = []
    leng_set = []
    maxleng = 1
    sentnum_per_dialogue = []
    while(i < len(minibatch)): #almost equal to BATCH_SIZE
        j = 0
        temp = []
        sentnum_per_dialogue.append(len(minibatch[i].Text)-1)
        #save sentnum per dialogue
        while(j < len(minibatch[i].Text)-1):
            sent, leng = sent_loader(minibatch[i].Text[j])  
            #convert text to word_list, word_list_length
            leng_set.append(leng)
            if leng > maxleng:
                maxleng = leng

            temp.append(sent)
            j = j + 1
        stop_tag = ["<stop_tag>"]
        temp.append(stop_tag) #append stoptag per dialogue
        new_batch.append(temp)
        i = i + 1

    i = 0
    while (i < len(minibatch)): #almost equal to BATCH_SIZE
        j = 0
        while (j < len(new_batch[i])):

            while(len(new_batch[i][j]) < maxleng):
                new_batch[i][j] = np.append(new_batch[i][j],"<pad>")

            j = j + 1
        i = i + 1
    #batch * sentnum * (word_list+pad) -> new_batch
    #batch * sentnum * (word_list_length) -> leng_set
    #batch * (sentnum) -> sentnum_per_dialogue
    print("max sentence in this batch:",maxleng)
    return new_batch, leng_set ,sentnum_per_dialogue


wv_model = word2vec.Word2Vec(size = 100, window = 5, min_count = 5, workers = 4)
wv_model = word2vec.Word2Vec.load('/home/jongsu/Desktop/dialogue_wv')

def numerize_sent(sent, len_sent):
    i = 0
    n_sent = []
    while(i < len_sent):
        if(sent[i] == '<pad>'):
            n_sent.append(np.zeros(100))
        else:
            try:
                n_sent.append(wv_model.wv[sent[i]])
            except:

                n_sent.append(np.zeros(100))

        i = i + 1
    return n_sent

def batch_numerical(sent_set):
    numeric_batch = []#numerized batch
    i = 0
    while(i < len(sent_set) ): #BATCH_SIZE
        dial = []#numerized dialogue
        j = 0
        while(j < len(sent_set[i])): #per dialogue
            dial.append(numerize_sent(sent_set[i][j], len(sent_set[i][j]))) #numerized_sentence
            j = j + 1
        numeric_batch.append(dial)
        i = i + 1
    #batch * sent_num * sent_leng * wv -> numeric_batch
    return numeric_batch

def select_sent_output(new_sent, all_seq_len):
    remake = torch.transpose(new_sent, 0, 1)
    i = 0
    while(i < len(remake)):
        if i == 0:
            tensor = remake[0][all_seq_len[0]-1].view(1,200)
        else:
            tensor = torch.cat((tensor, remake[i][all_seq_len[i]-1].view(1,200)), 0)
        i = i + 1
    return tensor

def make_batch2sent(new):
    for_sentmodel = []
    batchnum = 0
    while (batchnum < len(new)): #BATCH_SIZE
        for_sentmodel = for_sentmodel + new[batchnum]
        batchnum = batchnum + 1
    sentbatch_len = len(for_sentmodel)
    #batch * sent_num * sent_leng * wv -> all_sent_num * sent_leng * wv
    return sentbatch_len, for_sentmodel

def pad_dial(last_v):
    leng_set = []
    i = 0
    last_seq_vec = torch.ones([1,100], dtype=torch.float32, requires_grad=False).cuda()#stoptag

    while(i < len(last_v)):#BATCH_SIZE
        
        leng_set.append(len(last_v[i]))#sentence num

        #pad_vec = torch.cat((last_v[i],last_seq_vec),dim=0)#append stoptag vector
        #if i == 0:
        #    my_vec = pad_vec.unsqueeze(0)
        #else:
        #    my_vec = torch.cat((my_vec,pad_vec.unsqueeze(0)))
        i = i + 1
    padded_dial = pad_sequence(last_v, batch_first = True)#append padtag vector
    print('max_dialogue_length',len(padded_dial[0]))
    
    return padded_dial, leng_set

def sent_loader(sentence): #pre_process per sentence
    result = []
    for elem in sentence.split(' '):
        if elem != '':
            result = np.append(result, elem)
    return result,len(result)

def pad_cat_tag(emotion, act): 
    i = 0
    new_tag = []
    while(i < len(emotion)): #BATCH_SIZE
        emo, lenge = sent_loader(emotion[i][0])
        ac, lenga = sent_loader(act[i][0])
        j = 0
        inte = []
        inte.append(tag_to_ix['start_tag']) #append stop tag
        while(j < len(emo)): #sent length
            inte.append(int(emo[j]) * 4 + int(ac[j]))
            j = j + 1
        inte.append(tag_to_ix['stop_tag']) #append stop tag
        torch_inte = torch.tensor(inte)
        new_tag.append(torch_inte) #str to int
        i = i + 1

            
    padded_tag = pad_sequence(new_tag, batch_first = True, padding_value = tag_to_ix['pad_tag'])
    
    #emotion+action string -> emotion+action numb + padding
    #batch*tag
    return padded_tag

def make_mask(leng):
    '''
    make one-hot vector of mask from lengset
    '''
    var = np.zeros(shape = (len(leng), leng[0]+1)) #len(leng) = BATCH_SIZE, leng[0]+1= largest dialogue + stop
    i = 0
    while(i < len(leng)):#BATCH_SIZE
        j = 0
        while(j < leng[0]+1): 
            if(j <= leng[i]): # <= stop tag
                var[i][j] = 1
            j = j + 1
        
        i = i + 1
    return var

def log_sum_exp(x):
    max_score, _ = torch.max(x, -1)
    max_score_broadcast = max_score.unsqueeze(-1).expand_as(x)
    return max_score + torch.log(torch.sum(torch.exp(x - max_score_broadcast), -1))


In [8]:
class Example(object):
    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()
        for key, vals in fields.items():
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                                 "the input data".format(key))
            if vals is not None:
                if not isinstance(vals, list):
                    vals = [vals]
                for val in vals:
                    name, field = val
                    setattr(ex, name, field.preprocess(data[key]))
        return ex

In [9]:
class Dataset(torch.utils.data.Dataset):
    """Defines a dataset composed of Examples along with its Fields.
    Attributes:
        sort_key (callable): A key to use for sorting dataset examples for batching
            together examples with similar lengths to minimize padding.
        examples (list(Example)): The examples in this dataset.
        fields (dict[str, Field]): Contains the name of each column or field, together
            with the corresponding Field object. Two fields with the same Field object
            will have a shared vocabulary.
    """
    sort_key = None

    def __init__(self, examples, fields, filter_pred=None):
        """Create a dataset from a list of Examples and Fields.
        Arguments:
            examples: List of Examples.
            fields (List(tuple(str, Field))): The Fields to use in this tuple. The
                string is a field name, and the Field is the associated field.
            filter_pred (callable or None): Use only examples for which
                filter_pred(example) is True, or use all examples if None.
                Default is None.
        """

        self.examples = examples

        self.fields = dict(fields)

        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]
        self.pp = tuple(d for d in self.examples if d is not None)


    @classmethod
    def splits(cls, path=None, root='.data', train=None, **kwargs):

        train_data = cls(os.path.join(path, train), **kwargs)
        #print(train_data.examples) #여기엔 field example둘다 들어있음
        #print(tuple(d for d in train_data if d is not None)) #여기엔 example만 나열된 튜플이됨
        return tuple(d for d in train_data if d is not None)

    def split(self, split_ratio=0.7, stratified=False, strata_field='label',
              random_state=None):
        """Create train-test(-valid?) splits from the instance's examples.
        Arguments:
            split_ratio (float or List of floats): a number [0, 1] denoting the amount
                of data to be used for the training split (rest is used for validation),
                or a list of numbers denoting the relative sizes of train, test and valid
                splits respectively. If the relative size for valid is missing, only the
                train-test split is returned. Default is 0.7 (for th train set).
            stratified (bool): whether the sampling should be stratified.
                Default is False.
            strata_field (str): name of the examples Field stratified over.
                Default is 'label' for the conventional label field.
            random_state (int): the random seed used for shuffling.
        Returns:
            Tuple[Dataset]: Datasets for train, validation, and
                test splits in that order, if the splits are provided.
        """
        train_ratio, test_ratio, val_ratio = check_split_ratio(split_ratio)

        # For the permutations
        rnd = RandomShuffler(random_state)
        if not stratified:
            train_data, test_data, val_data = rationed_split(self.examples, train_ratio,
                                                             test_ratio, val_ratio, rnd)
        else:
            if strata_field not in self.fields:
                raise ValueError("Invalid field name for strata_field {}"
                                 .format(strata_field))
            strata = stratify(self.examples, strata_field)
            train_data, test_data, val_data = [], [], []
            for group in strata:
                # Stratify each group and add together the indices.
                group_train, group_test, group_val = rationed_split(group, train_ratio,
                                                                    test_ratio, val_ratio,
                                                                    rnd)
                train_data += group_train
                test_data += group_test
                val_data += group_val

        splits = tuple(Dataset(d, self.fields)
                       for d in (train_data, val_data, test_data) if d)

        # In case the parent sort key isn't none
        if self.sort_key:
            for subset in splits:
                subset.sort_key = self.sort_key
        return splits

    def __getitem__(self, i):
        return self.examples[i]

    def __len__(self):
        try:
            return len(self.examples)
        except TypeError:
            return 2 ** 32

    def __iter__(self):
        for x in self.examples:
            yield x

    def __getattr__(self, attr):
        if attr in self.fields:
            for x in self.examples:
                yield getattr(x, attr)

In [10]:
class my_TabularDataset(Dataset):

    def __init__(self, path,  fields,  **kwargs):


        with open(path, encoding="utf8") as f:
            for line in f:
                examples = [Example.fromdict(per_data, my_fields) for per_data in json.loads(line)]


        if isinstance(fields, dict):
            fields, field_dict = [], fields
            for field in field_dict.values():
                if isinstance(field, list):
                    fields.extend(field)
                else:
                    fields.append(field)


        super(my_TabularDataset, self).__init__(examples, fields, **kwargs)

In [11]:
class makesent_gru(nn.Module):
    def __init__(self, hidden_size, bidirectional):
        super(makesent_gru, self).__init__()
        self.hidden_size = hidden_size
        if (bidirectional == True):
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.gru = nn.GRU(100, 100, bidirectional=bidirectional)

    def forward(self, char, h0):
        gru_out, h0 = self.gru(char, h0)
        return gru_out, h0

    def initHidden(self):
        return torch.zeros(self.bidirectional, 1, self.hidden_size, device=device, requires_grad=False)

In [12]:
class last_sent_net(nn.Module):
    def __init__(self):
        super(last_sent_net, self).__init__()
        self.lastnet = nn.Linear(200, 100)

    def forward(self, last_hidden_state):
        last_w = self.lastnet(last_hidden_state)
        return last_w

In [13]:
class BiGru_CRF(nn.Module):
    def __init__(self, tag_to_ix, hidden_dim):
        super(BiGru_CRF, self).__init__()

        self.gru = nn.GRU(100, 100, bidirectional=True)
        self.hidden2tag = nn.Linear(200, 31)
        self.transitions = nn.Parameter(torch.randn(31, 31))# [a,b] trans from b to a
        self.transitions.data[0, :] = -10000 #all to start
        self.transitions.data[:, 29] = -10000 #stop to all
        self.transitions.data[:, 30] = -10000 #pad to all
        self.transitions.data[30][30] = 0 #pad to pad
        self.transitions.data[29][30] = 0 #stop to pad
        
        self.hidden_dim = hidden_dim
        self.tag_to_ix = tag_to_ix
        

    def init_hidden(self,batch):
        return Variable(torch.zeros(2, batch, 100).cuda(),requires_grad=False)
    
    def _get_gru_features(self, batch, sentence_set):
        
        hidden = self.init_hidden(batch)
        gru_out, hidden = self.gru(sentence_set, hidden)

        gru_feats = self.hidden2tag(gru_out)

        return gru_feats

    def for_score(self, pre_mask, feats):
        score = Variable(torch.zeros((BATCH_SIZE, 31)).fill_(-10000.).cuda(),requires_grad=False)
        score[:, self.tag_to_ix['start_tag']] = 0. #start to all is 0
        
        mask = Variable(torch.Tensor(pre_mask).cuda(),requires_grad=False)
        
        for t in range(feats.size(0)):  # 안에서 연산하는데이터들은 batch*featuresize*featuresize
            
            mask_t = mask[:, t].unsqueeze(-1).expand_as(score) #batch_size -> batch_size*featuresize
            
            score_t = score.unsqueeze(1).expand(-1, *self.transitions.size()) #batch_size*f -> batch_size*f*f
            
            emit = feats[t].unsqueeze(-1).expand_as(score_t) #b*f-> b*f*f
            
            trans = self.transitions.unsqueeze(0).expand_as(score_t) #b*f*f
            
            score_t = log_sum_exp(score_t + emit + trans)
            
            score = score_t * mask_t + score * (1 - mask_t) #no updating in masked score,all b*f

        score = log_sum_exp(score)
        return score

    def cal_score(self, mask, feats, tag):
        score = Variable(torch.FloatTensor(BATCH_SIZE).fill_(0.).cuda(),requires_grad=False)
        
        temp_tag = Variable(tag.cuda(),requires_grad=False)

        mask_tensor = torch.transpose(torch.FloatTensor(mask), 0, 1) #seq*batch
        mask_tensor = Variable(mask_tensor.cuda(), requires_grad=False)
        
        
        for i, feat in enumerate(feats): #seq*batch*feat->batch*feat
            
            transit = torch.cat(
                [torch.tensor([self.transitions[temp_tag[batch][i + 1], temp_tag[batch][i]]]) for batch in range(BATCH_SIZE)])
            
            transit = transit.cuda()
            
            transit = transit * mask_tensor[i] #batch*batch->batch
            
            emit = torch.cat([feat[batch][temp_tag[batch][i + 1]].view(1, -1) for batch in range(BATCH_SIZE)]).squeeze(1)

            emit = emit * mask_tensor[i]#batch*batch->batch
            
            score = score + transit + emit

        return score

    def neg_log_likelihood(self, mask, sentence, tags, batch):

        feats = self._get_gru_features(batch, sentence)
   
        forward_score = self.for_score(mask, feats)

        gold_score = self.cal_score(mask, feats, tags)

        return forward_score - gold_score

    def _viterbi_decode(self, mask, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = Variable(torch.full((1, 31), -10000.).cuda(),requires_grad=False)
        init_vvars[0][0] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars

        for i, feat in enumerate(feats):
            if mask[i] == 0:
                print('breaked')
                break
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(31):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))

            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        terminal_var = forward_var + self.transitions[self.tag_to_ix['stop_tag']]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == 0  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def forward(self,batch, dummy_input):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_gru_features(batch,dummy_input[1])
        
        
        
        lstm_feats = torch.transpose(lstm_feats,0,1)
        mask = dummy_input[0]

        
        
        # Find the best path, given thhttps://github.com/kkugosu/7proue features.
        score, tag_seq = self._viterbi_decode(mask[5], lstm_feats[5])
        return score, tag_seq
    

In [14]:
def all_preprocess(sent, lasent, batch_d):

    batch_data = sorted(batch_d, key = lambda  x: -len(x.Text))
    #sorted with dialogue length
    #print(batch_data[0].Text)
    #######################################################
    emotion_set = []
    action_set = []
    i = 0
    while(i < len(batch_data)): #equals to BATCH_SIZE except last dataset
        emotion_set.append(batch_data[i].labels_1)
        action_set.append(batch_data[i].labels_2)
        i = i + 1

    new_tag = pad_cat_tag(emotion_set, action_set)
    #batch*tag
    new_tag = Variable(new_tag.cuda(), requires_grad=False)

    
    #####################################################tag_preprocess
    
    
    new, all_seq_len, sentnum_per_batch = pad_batch(batch_data)
    #batch * sentnum * (word_list+pad) -> new
    #batch * sentnum * (word_list_length) -> all_seq_len
    #batch * (sentnum) -> sentnum_per_batch

    
    new2 = batch_numerical(new)
    #batch * sent_num * sent_leng * wv -> new2

    
    sentbatch_len, for_sentmodel = make_batch2sent(new2)
    #batch * sent_num * sent_leng * wv -> all_sent_num(new_batch) * sent_leng * wv
    
    hidden_state = torch.tensor(np.zeros((2, sentbatch_len, 100)), dtype=torch.float, device= device, requires_grad=False)
    for_sentmodel2 = torch.tensor(np.transpose(for_sentmodel, [1, 0, 2]), dtype=torch.float, device= device, requires_grad=False)
    
    new_sent, hidden_state = sent(for_sentmodel2, hidden_state)
    
    ##################################################first network
    
    out_sent_set = select_sent_output(new_sent, all_seq_len)
    #select last state according to masking
    
    pre_crf_gru = lasent(out_sent_set) #linear layer
    
    #################################################sec network
    #all_sent_num(new_batch) * sent_leng * wv -> all_sent_num(new_batch) * wv
    
    last_var = torch.split(pre_crf_gru, sentnum_per_batch)
    #all_sent_num(new_batch) * wv -> batch * sent_num * wv

    new_dial, dial_leng = pad_dial(last_var)
    #batch * sent_num * wv -> batch * (sent_num + pad) * wv
    #save dial_leng for masking
    
    new_dial = torch.transpose(new_dial, 0, 1)
    
    
    return new_dial, new_tag, dial_leng

In [17]:
my_fields={'dial': ('Text', data.Field(sequential=True)),
        'emo': ('labels_1', data.Field(sequential=False)),
        'act': ('labels_2', data.Field(sequential=False))}

train = my_TabularDataset.splits(path = Desktop_path, train = 'full_data.json',
                          fields=my_fields) #train에 fields라는 어트리뷰트가 없다는데?

train = sorted(train, key = lambda  x: cal_dialogue(x))
optimizer1 = optim.SGD(sent_to_vextor_bigru_net.parameters(), lr=0.00001, weight_decay=1e-4)
optimizer2 = optim.SGD(sent_to_vector_linear_net.parameters(), lr=0.00001, weight_decay=1e-4)
optimizer3 = optim.SGD(my_grucrf_model.parameters(), lr=0.00001, weight_decay=1e-4)


In [18]:

k = 0
for batch_data in batchload(train, repeat=True, BATCH_SIZE = BATCH_SIZE ):
    #load txt data from jsonfile
    sent_to_vextor_bigru_net.zero_grad()
    sent_to_vector_linear_net.zero_grad()
    my_grucrf_model.zero_grad()

    new_dial, new_tag, dial_leng = all_preprocess(sent_to_vextor_bigru_net, sent_to_vector_linear_net, batch_data)
    #load batch*(dialogue_length*sent_vec(float)) -> new_dial
    #load batch*tag -> new_tag
    #load batch * dial_leng

    loss = my_grucrf_model.neg_log_likelihood(make_mask(dial_leng), new_dial, new_tag, BATCH_SIZE)

    #print(loss)
    batch_loss = torch.sum(loss)
    print(batch_loss)
    batch_loss.backward(retain_graph=True)

    optimizer1.step()
    optimizer2.step()
    optimizer3.step()

    unuselist = [new_dial, new_tag, dial_leng]
    del unuselist
    
    
    #torch.save(a.state_dict(),'./log_dir/first.ckpt')
    #torch.save(b.state_dict(),'./log_dir/sec.ckpt')
    #torch.save(my_grucrf_model.state_dict(),'./log_dir/third.ckpt')
    
    #dummy_input = [make_mask(dial_leng),new_dial]
    #print(my_grucrf_model(dummy_input))
    
    #with SummaryWriter(comment='crf_gru') as w:
    #    w.add_graph(my_grucrf_model, (dummy_input, ))

    
    k = k + 1
    print(k)
    if k%100 == 10:
        torch.save(sent_to_vextor_bigru_net.state_dict(),'./first.pth')
        torch.save(sent_to_vector_linear_net.state_dict(),'./sec.pth')
        torch.save(my_grucrf_model.state_dict(),'./third.pth')
        print(new_tag[5])
        dummy_input = [make_mask(dial_leng),new_dial]
        print(my_grucrf_model(100,dummy_input))
        print(loss)
    if k == 3000:
        break


max sentence in this batch: 7


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 5 and 11 in dimension 1 at /opt/conda/conda-bld/pytorch_1524580978845/work/aten/src/THC/generic/THCTensorMath.cu:111

In [ ]:
print(os.environ.get('http_proxy'))

In [ ]:
tag_to_ix = {'start_tag':0,'stop_tag':29,'pad_tag':30}
tag_to_ix['start_tag']+tag_to_ix['stop_tag']

In [ ]:
#k = 0
#vis = visdom.Visdom()
#vis.text('Hello, world!')
#vis.image(np.ones((3, 10, 10)))
[2, 21,2, 21, 2,21,2, 21,25, 3,  3, 4,  29]
[2, 1, 2, 1, 2, 1, 2, 1, 17, 20, 17, 20]


[1,26, 3, 4, 26,1,  2, 1, 2,1,  17,1,  29]
[1, 2, 1, 1, 2, 1, 2, 3, 2, 1, 17, 20]